### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [3]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

In [4]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

### Kalman Filter

In [5]:
def kalman_filter(z, Q=1e-4, R=1e-2):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0
    for k in range(1, n):
        # previsão
        xhatminus = xhat[k-1]
        Pminus = P[k-1] + Q
        # atualização
        K = Pminus / (Pminus + R)
        xhat[k] = xhatminus + K * (z[k] - xhatminus)
        P[k] = (1 - K) * Pminus
    return xhat

In [6]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [7]:
kalman = kalman_filter(confidence_fusion)

# classe final a partir do filtrado
cls = (kalman >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [8]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'APMR':<6} {'MFSR':<6} {'IDG':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*75}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_apmr_edge[i]:<6.2f} {radnet_input_mfsr_edge[i]:<6.2f} {radnet_input_idg_global[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   APMR   MFSR   IDG    Classe   Status   Confiança 
---------------------------------------------------------------------------
-60    5.5   78     0.53   0.68   0.72   0.58   0        Ruim     0.59      
-34    2.0   38     0.34   0.57   0.68   0.67   1        Bom      0.77      
-90    6.9   42     0.57   0.76   0.83   0.74   1        Bom      0.62      
-85    3.2   48     0.59   0.84   0.92   0.73   1        Bom      0.59      
-45    3.3   12     0.07   0.79   0.88   0.71   1        Bom      0.66      
-90    6.9   86     0.67   0.48   0.55   0.64   1        Bom      0.59      
-49    2.4   35     0.25   0.84   0.88   0.75   1        Bom      0.65      
-36    1.1   21     0.18   0.47   0.57   0.66   1        Bom      0.66      
-62    5.9   31     0.15   0.80   0.89   0.82   1        Bom      0.68      
-77    8.8   84     0.75   0.78   0.87   0.76   1        Bom      0.65      
-46    1.7   22     0.26   0.92   0.93   0.72   1        Bom      0.69      


### Save

In [9]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
